# Porto Seguro - Initial submission

The data has already been explored. So now I will focus on feature engineering, model trainning and submission.

In [2]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt

import numpy as np
import seaborn as sns

#import joypy
import re
#from IPython.display import display, HTML
#import ipywidgets as widgets # for later


pd.options.display.float_format = '{:.2f}'.format
rc={'savefig.dpi': 75, 'figure.autolayout': False, 'figure.figsize': [12, 8], 'axes.labelsize': 12,\
   'axes.titlesize': 12, 'font.size': 12, 'lines.linewidth': 2.0, 'lines.markersize': 8, 'legend.fontsize': 12,\
   'xtick.labelsize': 12, 'ytick.labelsize': 12}

sns.set(style="darkgrid", color_codes=True,rc=rc)

In [83]:
train = pd.read_csv('train.csv')

In [4]:
test = pd.read_csv('test.csv')

Separating variables by type.

In [186]:
def MetaDataTypes(df,types_var=['cat','bin','target']):
    # Classifying the variables in the data
    variables = []
    vartype = {}
    for variable in df.columns:
        for types in types_var:
            ty = "None"
            if df[variable].dtype == int:
                tybin = "ordinal"
            elif df[variable].dtype == float:
                tybin = "continuous"
            match = re.search('^.*'+types+'.*$',variable)
            if match:
                ty = types
                if re.search('^.*bin.*$',variable):
                    tybin='binary'
                if re.search('^.*cat.*$',variable):
                    tybin='categorical'
                if 'target' in variable:
                    tybin = 'binary'
                break

            
        variables.append([variable,ty,tybin])
        
    variablesdf = pd.DataFrame(variables,columns=['name','type','bin'])
    
    for i in ['ordinal','continuous','binary','categorical']:
        vartype[i]=variablesdf.name[(variablesdf.bin==i)]

    # Creating dataframe containing variables
    return variablesdf,vartype

In [187]:
types_var = ['ind','reg','car', 'calc','target','id']
variablesdf,vartype = MetaDataTypes(train,types_var)

## Transforming categorical variables

In [197]:
# Code from https://github.com/felipeeeantunes/udacity_live

# In this code, it seems it is not droping the original variables
one_hot = {c: list(train[c].unique()) for c in train.columns if c not in ['id','target']}
def OHE_by_unique(train, one_hot, limit):
    
    #ONE-HOT enconde features with more than 2 and less than 'limit' unique values
    df = train.copy()
    for c in one_hot:
        if len(one_hot[c])>2 and len(one_hot[c]) < limit:
            #print(c,one_hot[c])
            for val in one_hot[c]:
                df[c+'_oh_' + str(val)] = (df[c].values == val).astype(np.int)
            df = df.drop(c,axis=1) # Davi Barreira - 5/11/2017
    return df

In [205]:
# My own function - It is less efficient

def Create_OHE(dataframe,variables,limit):
    df = dataframe.copy()
    for variable in variables:
        if len(pd.unique(train[variable])) < limit and len(pd.unique(train[variable])) > 2:
            columnsv = [variable + '_ohe' + str(value) for value in pd.unique(train[variable]).tolist()]
            dummy = pd.get_dummies(train[variable])
            dummy.columns = columnsv
            df = df.drop(variable,axis=1).copy().join(dummy)
            #print(variable,columnsv)
            
        else:
            pass
    return df

In [206]:
train_ohe = Create_OHE(train,vartype['categorical'].values,6)
#Create_OHE(traindrop,traindrop.columns,5)

In [201]:
train_ohe.shape

(595212, 75)

In [76]:
variable = 'ps_ind_02_cat'
dummy = pd.get_dummies(train[variable])
dummy.columns = columnsv


In [89]:
train_ohe = train.drop(variable,axis=1).copy().join(dummy)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
X = np.array(train)
y = np.array(y)
folds = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2017).split(X, y))

In [ ]:
def cross_val_model(X,y, model, n_splits=3):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import cross_val_score
    X = np.array(X)
    y = np.array(y)


    folds = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2017).split(X, y))

    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_holdout = X[test_idx]
        y_holdout = y[test_idx]

        print ("Fit %s fold %d" % (str(model).split('(')[0], j+1))
        model.fit(X_train, y_train)
        cross_score = cross_val_score(model, X_holdout, y_holdout, cv=3, scoring='roc_auc')
        print("    cross_score: %.5f" % cross_score.mean())  